In [1]:
from openpyxl import load_workbook
import re
wb = load_workbook("CMv2_TM4C_controller_pin_specs.xlsx")
print( wb)
ws = wb.active


In [2]:
for row in ws.iter_rows(min_row=9, max_col=6, max_row=136):
    print (row[0].value,row[1].value,row[5].value)

1 AIN15 CUR_V_4V0
2 AIN14 CUR_V_M3V3
3 AIN13 CUR_V_12V
4 AIN12 V_F2_VCCAUX
5 PQ0 /I2C_RESET_PWR
6 PQ1 /I2C_RESET_CLOCKS
7 VDD V_M3V3
8 VDDA V_M3V3
9 VREFA+ MCU_VREFA
10 GNDA GND
11 PQ2 /I2C_RESET_F2_OPTICS
12 AIN0 V_12V
13 AIN1 V_M3V3
14 AIN2 V_3V3
15 AIN3 V_4V0
16 VDD V_M3V3
17 GND GND
18 AIN16 CUR_F1_VCCAUX
19 AIN17 CUR_F2_VCCAUX
20 AIN18 F1_TEMP_DIODE
21 AIN19 F2_TEMP_DIODE
22 PC7 PG_4V0
23 PC6 F2_CFG_START
24 PC5 SM_ZYNQ_GPIO2
25 PC4 SM_ZYNQ_GPIO1
26 VDD V_M3V3
27 PQ3 /I2C_RESET_F1_OPTICS
28 VDD V_M3V3
29 PH0 /I2C_RESET_FPGAS
30 PH1 SM_SOFT_PWR_EN
31 PH2 PG_3V3
32 PH3 PG_1V8
33 U0Rx SM_ZYNQ_TX
34 U0Tx SM_ZYNQ_RX
35 PA2 SM_ZYNQ_I2C_SCL
36 PA3 SM_ZYNQ_I2C_SDA
37 U3Rx SM_IPMC_TX
38 U3Tx SM_IPMC_RX
39 VDD V_M3V3
40 PA6 SM_CPLD_GPIO0
41 PA7 SM_CPLD_GPIO1
42 PF0 PG_F1_INT_A
43 PF1 PG_F1_INT_B
44 PF2 PG_F1_AVCC
45 PF3 PG_F1_AVTT
46 PF4 PG_F1_VCCAUX
47 VDD V_M3V3
48 GND GND
49 I2C1SCL I2C_SCL_POWER
50 I2C1SDA I2C_SDA_POWER
51 I2C2SCL I2C_SCL_CLOCKS
52 I2C2SDA I2C_SDA_CLOCKS
53 I2C3SCL I2C_

In [3]:
class gpio_pin_t:
    name = ""
    pin = -1
    port = ""
    local_pin = -1 
    is_input = True
    def __init__(self, name, pin, port, local_pin, is_input):
        self.pin = pin
        self.name = name
        self.port = port
        self.local_pin = local_pin
        self.is_input = is_input
        
class pin_t:
    name = ""
    pin = -1
    def __init__(self, name, pin) :
        self.pin = pin
        self.name = name
        

In [7]:
pinsl=[]
allpinsl = []

for row in ws.iter_rows(min_row=9, max_col=6, max_row=136):
    
    pin = row[0].value
    is_input = True 
    if ( row[2].value == "O"):
        is_input = False
    signame=re.split(", |: ",row[5].value)[0]
    signame = re.sub('/', '_', signame)
    # if there is a newline in signam, then it needs to be split based on the newline
    if ( re.search("\n", signame) != [] ) :
        signame = re.split("\n", signame)[0].replace("hiv","")
    print ("before: %s, after: %s" % (row[5].value, signame))
    #print  row[4].value, re.split(r'GPIO port +([A-Z]+) bit ([0-9]+)', row[4].value)
    #print pin, signame, gpio_port, gpio_pin
    #print "#define ", signame, pin, "// Port, local pin=", gpio_port, gpio_pin, ", input=", is_input
    is_not_gpio = row[4].value.find("GPIO")
    allpinsl.append(pin_t(name=signame, pin=pin))
    if ( is_not_gpio ):
        #print "not gpio ->",row[4].value, row[5].value
        continue
    [undef, gpio_port, gpio_pin, undef ]  = re.split(r'GPIO port +([A-Z]+) bit ([0-9]+)', row[4].value)
    pinsl.append(gpio_pin_t(name=signame,pin=pin, port=gpio_port, local_pin = gpio_pin, is_input=is_input))

header = open("tmp_pin_defs.h", "w")
#print len(pinsl)
for p in pinsl:
    header.write("#define %s %s // port, local_pin = %s %s \n" % (p.name, p.pin, p.port, p.local_pin) )
header.close()

body = open("tmp_pin_defs.c", "w")
body.write( "\n\n\n")
body.write( "switch(pin):\n")
body.write( "{\n")
for p in pinsl:
    body.write("\tcase %s :\n\t{\n" % p.name)
    body.write("\t\tgpio_port = GPIO_PORT%s_BASE;\n" % p.port)
    body.write("\t\tgpio_pin  = GPIO_PIN_%s;\n" % p.local_pin)
    body.write("\t\tbreak;\n\t}\n")
body.write("\tdefault:\n")
body.write("\t\tgpio_port = -1;\n")
body.write("\t\tgpio_pin  = -1;\n")
body.write("\t\tbreak;\n")
body.write("}\n")


before: CUR_V_4V0, after: CUR_V_4V0
before: CUR_V_M3V3, after: CUR_V_M3V3
before: CUR_V_12V, after: CUR_V_12V
before: V_F2_VCCAUX, after: V_F2_VCCAUX
before: /I2C_RESET_PWR, after: _I2C_RESET_PWR
before: /I2C_RESET_CLOCKS, after: _I2C_RESET_CLOCKS
before: V_M3V3, after: V_M3V3
before: V_M3V3, after: V_M3V3
before: MCU_VREFA, after: MCU_VREFA
before: GND, after: GND
before: /I2C_RESET_F2_OPTICS, after: _I2C_RESET_F2_OPTICS
before: V_12V, after: V_12V
before: V_M3V3, after: V_M3V3
before: V_3V3, after: V_3V3
before: V_4V0, after: V_4V0
before: V_M3V3, after: V_M3V3
before: GND, after: GND
before: CUR_F1_VCCAUX, after: CUR_F1_VCCAUX
before: CUR_F2_VCCAUX, after: CUR_F2_VCCAUX
before: F1_TEMP_DIODE, after: F1_TEMP_DIODE
before: F2_TEMP_DIODE, after: F2_TEMP_DIODE
before: PG_4V0, after: PG_4V0
before: F2_CFG_START, after: F2_CFG_START
before: SM_ZYNQ_GPIO2, after: SM_ZYNQ_GPIO2
before: SM_ZYNQ_GPIO1, after: SM_ZYNQ_GPIO1
before: V_M3V3, after: V_M3V3
before: /I2C_RESET_F1_OPTICS, after: _I2

2

In [8]:
body.write("const char* const pin_names[] =\n")
body.write("{\n")
for p in allpinsl:
    body.write("\"%s\",// %d \n"% (p.name, p.pin) )
body.write("};\n")

body.close()


In [16]:
f = open("/tmp/test.def", "w")

# x-macros version
for p in pinsl:
    direction = "true" # c-style boolean values
    if ( p.is_input == False ):
        direction = "false"
    f.write(f'   X({p.name}, {p.pin}, {p.port}, {p.local_pin}, {direction}) \n')
f.write("#undef X\n")
f.close()